# **Modelo de recomendación!**

https://www.kaggle.com/datasets/tamber/steam-video-games/

### **Load data**

In [3]:
# Importar librerias
import pandas as pd
ratings = pd.read_csv('../src/context/videogames.csv', sep=';', names=['UserId', 'VideoGame', 'Purchase', 'HoursPlayed'], index_col=False)

C:\Users\andres.tellez\AppData\Local\Temp\ipykernel_10168\2247618012.py:3: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  ratings = pd.read_csv('../src/context/videogames.csv', sep=';', names=['UserId', 'VideoGame', 'Purchase', 'HoursPlayed'], index_col=False)


In [4]:
ratings

,UserId,VideoGame,Purchase,HoursPlayed
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0
1,151603712,The Elder Scrolls V Skyrim,play,273.0
2,151603712,Fallout 4,purchase,1.0
3,151603712,Fallout 4,play,87.0
4,151603712,Spore,purchase,1.0
...,...,...,...,...
199993,128470551,Titan Souls,play,1.5
199994,128470551,Grand Theft Auto Vice City,purchase,1.0
199995,128470551,Grand Theft Auto Vice City,play,1.5
199996,128470551,RUSH,purchase,1.0


### **Analytics**

In [8]:
import sweetviz as sv

In [9]:
report = sv.analyze(ratings)
report.show_html('Report.html')

                                             |          | [  0%]   00:00 -> (? left)

Report Report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


- Tenemos un poco menos de 12,500 usuarios diferentes que han jugado más de 5,000 juegos diferentes.

Veo que hay dos diferentes eventos a los que la base de datos hace referencia. Aquellos juegos que han sido jugados y aquellos que han sido comprados. El recomendador que quiero crear esta un poco más del lado de dar juegos cuya probabilidad de ser jugados sea alta, en contrario, a recomendar juegas con alta probabilidad de ser comprados. Con esto en mente, voy a desahcerme de los comprados.

In [5]:
ratings = ratings[ratings['Purchase'] == 'play']

In [6]:
ratings

,UserId,VideoGame,Purchase,HoursPlayed
1,151603712,The Elder Scrolls V Skyrim,play,273.0
3,151603712,Fallout 4,play,87.0
5,151603712,Spore,play,14.9
7,151603712,Fallout New Vegas,play,12.1
9,151603712,Left 4 Dead 2,play,8.9
...,...,...,...,...
199989,128470551,Fallen Earth,play,2.4
199991,128470551,Magic Duels,play,2.2
199993,128470551,Titan Souls,play,1.5
199995,128470551,Grand Theft Auto Vice City,play,1.5


In [12]:
report = sv.analyze(ratings)
report.show_html('Play.html')

                                             |          | [  0%]   00:00 -> (? left)

Report Play.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


- El número de usuarios que han jugado bajó a 11,350, eso quiere decir que hay un 8% de los usuarios que han comprado algún juego pero aún no lo prueban. 
- Asimismo, el número de videjuegos jugados son 3,599, siendo Dota 2 el juego que más usuarios distintos han jugado (7% de cobertura). Asimismo, el segundo juego más Team Fortress con tan solo 3%.
- En promedio de las horas que se han jugado es de 49 horas. Y en total acumulando más de 3M de horas jugadas. Tambien es important notar que la varianza es de 52 horas, por lo que se distingue claramente que estamos trabajado con una base de datos que tiene unas horas jugadas muy aplias.

### **Model exploration**

#### **Intento 1**

**Horas jugadas como calificaciones:** este enfoque utiliza directamente el tiempo dedicado por un usuario a un juego como indicador de sus preferencias. Sin embargo, esto puede introducir un sesgo en los juegos que son inherentemente más largos o que los usuarios pueden dejar de ejecutar sin jugar activamente. Las calificaciones suelen indicar preferencia o calidad, no cuánto ha consumido un usuario.

In [7]:
ratings = ratings[['UserId', 'VideoGame', 'HoursPlayed']]

In [9]:
ratings

,UserId,VideoGame,HoursPlayed
1,151603712,The Elder Scrolls V Skyrim,273.0
3,151603712,Fallout 4,87.0
5,151603712,Spore,14.9
7,151603712,Fallout New Vegas,12.1
9,151603712,Left 4 Dead 2,8.9
...,...,...,...
199989,128470551,Fallen Earth,2.4
199991,128470551,Magic Duels,2.2
199993,128470551,Titan Souls,1.5
199995,128470551,Grand Theft Auto Vice City,1.5


In [10]:
from surprise import Dataset, Reader

# Convert dataframes to surprise datasets
reader = Reader(rating_scale=(0, ratings['HoursPlayed'].max()))
data = Dataset.load_from_df(ratings, reader)

In [11]:
from surprise import SVD
from surprise.model_selection import cross_validate

# Define the SVD algorithm object
algo = SVD()

# Run 5-fold cross-validation and then print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    11706.650811704.761911703.720811702.112011704.990811704.44731.4987  
MAE (testset)     11702.218411699.594711698.901111696.444611699.440611699.31991.8403  
Fit time          0.64    0.58    0.53    0.53    0.53    0.56    0.04    
Test time         0.07    0.06    0.10    0.06    0.11    0.08    0.02    


{'test_rmse': array([11706.65082225, 11704.76190885, 11703.72076515, 11702.11204802,
        11704.99079375]),
 'test_mae': array([11702.21839742, 11699.59470652, 11698.90110084, 11696.4445535 ,
        11699.4405581 ]),
 'fit_time': (0.6396520137786865,
  0.5846195220947266,
  0.5327537059783936,
  0.5277884006500244,
  0.5337843894958496),
 'test_time': (0.07241201400756836,
  0.05921578407287598,
  0.10343360900878906,
  0.05914449691772461,
  0.10626745223999023)}

El error absoluto medio (MAE) de 1700 sugiere que, en promedio, las predicciones del modelo están a 1700 horas de los valores reales. Esto parece bastante alto, especialmente considerando que es poco probable que alguien juegue un solo juego durante 1700 horas. Por lo tanto, es posible que el score no sea el adecuado.

#### **Intento 2**

**Percentiles por usuario:** Para cada usuario se calcula el percentil de horas jugadas para cada juego que este tiene. 

In [12]:
# Calculate the total hours played by each user
total_hours_by_user = ratings.groupby('UserId')['HoursPlayed'].sum()

# Join this data with your original DataFrame
ratings = ratings.join(total_hours_by_user, on='UserId', rsuffix='_total')

# Calculate the percentile of hours played for each game by the user
ratings['HoursPlayedPercentile'] = ratings.groupby('UserId')['HoursPlayed'].apply(lambda x: x.rank(pct=True))

C:\Users\andres.tellez\AppData\Local\Temp\ipykernel_10168\3348133536.py:8: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  ratings['HoursPlayedPercentile'] = ratings.groupby('UserId')['HoursPlayed'].apply(lambda x: x.rank(pct=True))


In [15]:
ratings

,UserId,VideoGame,HoursPlayed,HoursPlayed_total,HoursPlayedPercentile
1,151603712,The Elder Scrolls V Skyrim,273.0,439.9,1.000000
3,151603712,Fallout 4,87.0,439.9,0.961538
5,151603712,Spore,14.9,439.9,0.923077
7,151603712,Fallout New Vegas,12.1,439.9,0.884615
9,151603712,Left 4 Dead 2,8.9,439.9,0.846154
...,...,...,...,...,...
199989,128470551,Fallen Earth,2.4,426.9,0.312500
199991,128470551,Magic Duels,2.2,426.9,0.250000
199993,128470551,Titan Souls,1.5,426.9,0.156250
199995,128470551,Grand Theft Auto Vice City,1.5,426.9,0.156250


In [14]:
ratings[ratings['UserId']==151603712]

,UserId,VideoGame,HoursPlayed,HoursPlayed_total,HoursPlayedPercentile
1,151603712,The Elder Scrolls V Skyrim,273.0,439.9,1.000000
3,151603712,Fallout 4,87.0,439.9,0.961538
5,151603712,Spore,14.9,439.9,0.923077
7,151603712,Fallout New Vegas,12.1,439.9,0.884615
9,151603712,Left 4 Dead 2,8.9,439.9,0.846154
11,151603712,HuniePop,8.5,439.9,0.807692
13,151603712,Path of Exile,8.1,439.9,0.769231
15,151603712,Poly Bridge,7.5,439.9,0.730769
17,151603712,Left 4 Dead,3.3,439.9,0.692308
19,151603712,Team Fortress 2,2.8,439.9,0.653846


In [16]:
ratings_percentile = ratings[['UserId', 'VideoGame', 'HoursPlayedPercentile']]

In [17]:
from surprise import Dataset, Reader

# Convert dataframes to surprise datasets
reader = Reader(rating_scale=(0, ratings_percentile['HoursPlayedPercentile'].max()))
data = Dataset.load_from_df(ratings_percentile, reader)

In [18]:
from surprise import SVD
from surprise.model_selection import cross_validate

# Define the SVD algorithm object
algo = SVD()

# Run 5-fold cross-validation and then print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.2716  0.2764  0.2723  0.2744  0.2725  0.2734  0.0018  
MAE (testset)     0.2204  0.2245  0.2211  0.2229  0.2212  0.2220  0.0015  
Fit time          0.59    0.63    0.54    0.53    0.54    0.56    0.04    
Test time         0.08    0.06    0.11    0.06    0.06    0.07    0.02    


{'test_rmse': array([0.27161376, 0.2763901 , 0.27225085, 0.27444966, 0.27245805]),
 'test_mae': array([0.22040706, 0.22448295, 0.22105601, 0.22291862, 0.22122831]),
 'fit_time': (0.5890703201293945,
  0.6267013549804688,
  0.5373120307922363,
  0.5269269943237305,
  0.536555290222168),
 'test_time': (0.08160710334777832,
  0.06253552436828613,
  0.10611629486083984,
  0.0603029727935791,
  0.055977582931518555)}

El error absoluto medio (MAE) de 0,22 es mucho menor que el inicial de 1700, lo que supone una mejora. Esto significa que, en promedio, las predicciones del modelo están a 0,22 de los percentiles reales.

Esta comparación sugiere firmemente que usar percentiles de horas jugadas como calificaciones es una opción mucho mejor que usar las horas sin procesar para su conjunto de datos específico.

Ahora bien, la pregunta es que pasaria si alternativamente se usan los **percentiles de para cada videojuego**?

#### **Intento 3**

In [19]:
ratings['HoursPlayedPercentileVG'] = ratings.groupby('VideoGame')['HoursPlayed'].apply(lambda x: x.rank(pct=True))

C:\Users\andres.tellez\AppData\Local\Temp\ipykernel_10168\1738498441.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  ratings['HoursPlayedPercentileVG'] = ratings.groupby('VideoGame')['HoursPlayed'].apply(lambda x: x.rank(pct=True))


In [21]:
ratings[ratings['UserId']==151603712]

,UserId,VideoGame,HoursPlayed,HoursPlayed_total,HoursPlayedPercentile,HoursPlayedPercentileVG
1,151603712,The Elder Scrolls V Skyrim,273.0,439.9,1.000000,0.904727
3,151603712,Fallout 4,87.0,439.9,0.961538,0.790419
5,151603712,Spore,14.9,439.9,0.923077,0.775862
7,151603712,Fallout New Vegas,12.1,439.9,0.884615,0.404181
9,151603712,Left 4 Dead 2,8.9,439.9,0.846154,0.461298
11,151603712,HuniePop,8.5,439.9,0.807692,0.250000
13,151603712,Path of Exile,8.1,439.9,0.769231,0.578534
15,151603712,Poly Bridge,7.5,439.9,0.730769,0.750000
17,151603712,Left 4 Dead,3.3,439.9,0.692308,0.311275
19,151603712,Team Fortress 2,2.8,439.9,0.653846,0.422084


In [20]:
ratings

,UserId,VideoGame,HoursPlayed,HoursPlayed_total,HoursPlayedPercentile,HoursPlayedPercentileVG
1,151603712,The Elder Scrolls V Skyrim,273.0,439.9,1.000000,0.904727
3,151603712,Fallout 4,87.0,439.9,0.961538,0.790419
5,151603712,Spore,14.9,439.9,0.923077,0.775862
7,151603712,Fallout New Vegas,12.1,439.9,0.884615,0.404181
9,151603712,Left 4 Dead 2,8.9,439.9,0.846154,0.461298
...,...,...,...,...,...,...
199989,128470551,Fallen Earth,2.4,426.9,0.312500,0.833333
199991,128470551,Magic Duels,2.2,426.9,0.250000,0.442857
199993,128470551,Titan Souls,1.5,426.9,0.156250,0.750000
199995,128470551,Grand Theft Auto Vice City,1.5,426.9,0.156250,0.456522


In [22]:
ratings_percentile_VG = ratings[['UserId', 'VideoGame', 'HoursPlayedPercentileVG']]

In [23]:
from surprise import Dataset, Reader

# Convert dataframes to surprise datasets
reader = Reader(rating_scale=(0, ratings_percentile_VG['HoursPlayedPercentileVG'].max()))
data = Dataset.load_from_df(ratings_percentile_VG, reader)

In [24]:
from surprise import SVD
from surprise.model_selection import cross_validate

# Define the SVD algorithm object
algo = SVD()

# Run 5-fold cross-validation and then print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.2925  0.2932  0.2966  0.2954  0.2957  0.2947  0.0016  
MAE (testset)     0.2483  0.2485  0.2522  0.2511  0.2514  0.2503  0.0016  
Fit time          0.68    0.67    0.68    0.55    0.52    0.62    0.07    
Test time         0.14    0.07    0.08    0.11    0.06    0.09    0.03    


{'test_rmse': array([0.29246451, 0.29322739, 0.29664313, 0.29543722, 0.29574608]),
 'test_mae': array([0.24830294, 0.24848748, 0.25223628, 0.25111384, 0.25144587]),
 'fit_time': (0.6758608818054199,
  0.6676242351531982,
  0.6790890693664551,
  0.5495586395263672,
  0.5216846466064453),
 'test_time': (0.1445479393005371,
  0.07159042358398438,
  0.08056354522705078,
  0.10725545883178711,
  0.05757331848144531)}

En esta exploracion para un tercer score, vemos que el percentil basado en el videojuego empeoró en 3 puntos porcentuales. De esta manera se propone seguir mejorando el percentil basado en usuarios.

Se nota que el número de videojuegos baja de casi 200,000 a un poco menos de 30,000.

#### **Intento 4**

En este caso se va a intentar sacar algo parecido a lo que se logro con el percentil, vamos a coger el maximo tiempo que un jugador tiene para un videojuego y usar ese de referencia para saber cuanto ponerle a los otros.

In [28]:
# Ahora buscamos el juego con maximo tiempo jugado por cada usuario.
df_max_hours = ratings.groupby('UserId')['HoursPlayed'].max().reset_index()
df_max_hours.rename(columns={"HoursPlayed": "MaxHoursPlayed"}, inplace=True)

# Hacemos un left join con df original y el dataframe con maximos tiempos
ratings = pd.merge(ratings, df_max_hours, on='UserId', how="left")

# Creamos una nueva columna con el tiempo jugado de cada videojuego dividido por el tiempo máximo jugado por el usuario.
ratings['PlayedProportion'] = ratings['HoursPlayed'] / ratings['MaxHoursPlayed']


In [29]:
ratings[ratings['UserId']==151603712]

,UserId,VideoGame,HoursPlayed,HoursPlayed_total,HoursPlayedPercentile,HoursPlayedPercentileVG,MaxHoursPlayed,PlayedProportion
0,151603712,The Elder Scrolls V Skyrim,273.0,439.9,1.000000,0.904727,273.0,1.000000
1,151603712,Fallout 4,87.0,439.9,0.961538,0.790419,273.0,0.318681
2,151603712,Spore,14.9,439.9,0.923077,0.775862,273.0,0.054579
3,151603712,Fallout New Vegas,12.1,439.9,0.884615,0.404181,273.0,0.044322
4,151603712,Left 4 Dead 2,8.9,439.9,0.846154,0.461298,273.0,0.032601
5,151603712,HuniePop,8.5,439.9,0.807692,0.250000,273.0,0.031136
6,151603712,Path of Exile,8.1,439.9,0.769231,0.578534,273.0,0.029670
7,151603712,Poly Bridge,7.5,439.9,0.730769,0.750000,273.0,0.027473
8,151603712,Left 4 Dead,3.3,439.9,0.692308,0.311275,273.0,0.012088
9,151603712,Team Fortress 2,2.8,439.9,0.653846,0.422084,273.0,0.010256


In [30]:
ratings

,UserId,VideoGame,HoursPlayed,HoursPlayed_total,HoursPlayedPercentile,HoursPlayedPercentileVG,MaxHoursPlayed,PlayedProportion
0,151603712,The Elder Scrolls V Skyrim,273.0,439.9,1.000000,0.904727,273.0,1.000000
1,151603712,Fallout 4,87.0,439.9,0.961538,0.790419,273.0,0.318681
2,151603712,Spore,14.9,439.9,0.923077,0.775862,273.0,0.054579
3,151603712,Fallout New Vegas,12.1,439.9,0.884615,0.404181,273.0,0.044322
4,151603712,Left 4 Dead 2,8.9,439.9,0.846154,0.461298,273.0,0.032601
...,...,...,...,...,...,...,...,...
70475,128470551,Fallen Earth,2.4,426.9,0.312500,0.833333,291.0,0.008247
70476,128470551,Magic Duels,2.2,426.9,0.250000,0.442857,291.0,0.007560
70477,128470551,Titan Souls,1.5,426.9,0.156250,0.750000,291.0,0.005155
70478,128470551,Grand Theft Auto Vice City,1.5,426.9,0.156250,0.456522,291.0,0.005155


In [31]:
ratings_played_proportion = ratings[['UserId', 'VideoGame', 'PlayedProportion']]

In [32]:
from surprise import Dataset, Reader

# Convert dataframes to surprise datasets
reader = Reader(rating_scale=(0, ratings_played_proportion['PlayedProportion'].max()))
data = Dataset.load_from_df(ratings_played_proportion, reader)

In [33]:
from surprise import SVD
from surprise.model_selection import cross_validate

# Define the SVD algorithm object
algo = SVD()

# Run 5-fold cross-validation and then print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.2882  0.2864  0.2860  0.2838  0.2889  0.2867  0.0018  
MAE (testset)     0.1860  0.1855  0.1853  0.1846  0.1869  0.1857  0.0008  
Fit time          0.58    0.57    0.57    0.68    0.58    0.60    0.04    
Test time         0.06    0.06    0.06    0.07    0.06    0.06    0.01    


{'test_rmse': array([0.28819684, 0.28642205, 0.28597133, 0.28375962, 0.28890683]),
 'test_mae': array([0.1860238 , 0.1855388 , 0.18526665, 0.18463833, 0.18692892]),
 'fit_time': (0.5809621810913086,
  0.568824291229248,
  0.5745422840118408,
  0.675694465637207,
  0.5841524600982666),
 'test_time': (0.057381391525268555,
  0.05680203437805176,
  0.06224942207336426,
  0.07077884674072266,
  0.06262731552124023)}

### **Guardar modelo**

In [34]:
import pickle

with open('../models/my_model_proportion.pkl', 'wb') as file:
    pickle.dump(algo, file)

### **Probar modelo**

In [38]:
import pandas as pd 

user_id=128470551

n_recommendations=3

# Load the model
with open('my_model_proportion.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Load the game data
df = pd.read_csv('../src/context/videogames.csv', sep=';', names=['UserId', 'VideoGame', 'Purchase', 'HoursPlayed'], index_col=False)

# Find the games the user has not played yet
not_played = df.loc[~df['VideoGame'].isin(df.loc[df['UserId'] == user_id, 'VideoGame'])]['VideoGame'].unique()

# Predict ratings for all those games
predictions = [loaded_model.predict(user_id, game) for game in not_played]

# Sort the predictions by estimate in descending order and get the top n_recommendations
ranked_games = sorted(predictions, key=lambda x: x.est, reverse=True)[:n_recommendations]

# Reccomendations
recom = [pred.iid for pred in ranked_games]

C:\Users\andres.tellez\AppData\Local\Temp\ipykernel_10168\606490292.py:12: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv('../src/context/videogames.csv', sep=';', names=['UserId', 'VideoGame', 'Purchase', 'HoursPlayed'], index_col=False)


In [39]:
ranked_games

[Prediction(uid=128470551, iid='Dota 2', r_ui=None, est=0.9322866427003249, details={'was_impossible': False}),
 Prediction(uid=128470551, iid='Team Fortress 2', r_ui=None, est=0.8332556406004573, details={'was_impossible': False}),
 Prediction(uid=128470551, iid='Professional Farmer 2014', r_ui=None, est=0.7519322669607083, details={'was_impossible': False})]